In [ ]:
%pip install imageio==2.4.1
from ai2thor_colab import show_video
import ai2thor_colab
import tqdm
ai2thor_colab.start_xserver()
# get trajectories
import h5py
import numpy as np
from allenact.utils.inference import InferenceAgent
from transformers import T5Tokenizer
from frozen_lm_qa.flax_model import FlaxT5ForMemoryQuestionAnswering

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/haoz/allenact/projects/procthor-rearrangement/")
sys.path.append("/home/haoz/allenact/projects/procthor-rearrangement/ai2thor_rearrangement/")
from procthor_baseline_configs.one_phase.one_phase_rgb_clip_dagger_multi_node import (
    ProcThorOnePhaseRGBClipResNet50DaggerTrainMultiNodeConfig as Config,
)
from rearrange.environment import RearrangeTaskSpec

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = FlaxT5ForMemoryQuestionAnswering.from_pretrained("/home/haoz/t5x-frozen-lm-qa-base")

In [ ]:
config = Config()
task_sampler_args = config.train_task_sampler_args(process_ind=0, total_processes=2500)
task_sampler_args["epochs"] = 1
task_sampler_args["allowed_scenes"] = ["train_9404"]

In [ ]:
task_sampler = config.make_sampler_fn(**task_sampler_args)
task = task_sampler.next_task()

In [ ]:
question = tokenizer(["Is there a Drawer in the Bedroom?"], return_tensors="np")

In [ ]:
task = task_sampler.next_task()
frames = []
observations = task.get_observations()
agent = InferenceAgent.from_experiment_config(
        exp_config=config,
        checkpoint_path="/home/haoz/ckpt_jordi.pt",
        device="cuda:1",
    )
agent.reset()
# for action in tqdm.tqdm(actions):
while not task.is_done():
    action = agent.act(observations=observations)
    hid = agent.memory.tensor("rnn").detach().cpu().numpy()
    summary_ids = model.generate(hid, question["input_ids"]).sequences
    print(tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))
    frame = task.env.controller.last_event.frame
    observations = task.step(action).observation
    frames.append(frame)
show_video(frames, fps=5)